# 2021-12-12 oAN tests - bringing some imperfections in the model implementation to the fore

Results below are for commit: https://github.com/gmarkkula/COMMOTIONSFramework/commit/22fd40db4e6fb1cf827cf930f3c13402cab342df (Rerunning the code below for other commits is not likely/guaranteed to give the same results.)

After trying to understand why the `oAN` models exhibitied apparent pedestrian hesitation, I found the following, illustrated by the test run below:
* The `oAN` models have a natural tendency to slow down, even when there is no other road user; below `oVAoAN` averages 1.1 m/s between its starting boing and the conflict space, in the scenario where the car is already 3 s past the conflict space. Note that `oVAoSNv` shows an average of 1.3 m/s, i.e., the free speed of the base model. Iam not 100 % sure what causes this deceleration tendency in the `oAN` models. It might have something to do with the saturation effect of the model not being able to reverse, where it may, due to the value noise, first apply rather large decelerations which would have it reverse, and then it has to wait for a while at zero speed, until those predicted decelerations have cleared out of the "prediction buffer", before it can see a possibility of increasing speed toward the free speed again? 
    * So possibly one way of dealing with this would be to make sure that the predicted buffer of accelerations doesn't contain negative accelerations at future time points where the accelerations will in fact be zero because speed will have reached zero already?
    * Another way of dealing with it, without modifying those low-level parts of the model, would be to add something like the "car TTCA = -3 s" scenario belo to the probabilistic fits, and then measure hesitation against that baseline. That would add a bit of fitting time though.
* Another, more concerning matter, is the reason why `oVAoAN` is slowing down further when the car TTCA is at 8 s, and even further when it is at 80 s. (Again, `oVAoSNv` is behaving as one would expect, with a near-negligible slow-down in the 80 s scenario.) The reason the `oVAoAN` model does this seems to be the previously noted imperfection in `sc_scenario_helper.get_access_order_implications()` which doesn't always correctly consider the possibility of just accelerating to free speed to pass first/second. It seems in this particular case what is happening is that the pedestrian first reaches a low speed by chance due to the accumulator noise, and then when it considers speeding up again, if the speed change would bring it above its free speed, such that the "acceleration to go to free speed" is negative, the current simplified/imperfect check for the "just go to free speed" option for passing first fails, because it only looks for cases where passing first would mean accelerating at a faster rate than the rate estimated for passing just in front of the vehicle (which in the TTCA 80 s case often entails waiting just beyond the exit of the conflict space for quite a while, which is actually quite a severe imperfection). Possibly this is also exacerbated by above-mentioned "acceleration buffer" thing.
    * What would be needed to fix this is an accurate test of the option of just going to free speed, both for passing first and second. I think this should be done by adding a new function `sc_scenario_helper.get_free_acc_dist_travelled()` which takes initial condition, free acceleration and free acceleration time, and calculates how far the agent will have travelled in a given time (equal to the time until other agent arrives at / leaves the conflict space if testing for passing first / second). Needs to consider the possibility that the given time will elapse either during the time of acceleration to free speed, or after. 

Clearly the `oAN` tests have put emphasis on some shortcomings in the model implementation, which didn't lead to (as obvious) problems previously. I will think a bit further about it, but overall I believe the best approach is to:
* Sort out the "excessive negative accelerations in buffer" thing.
* Add the more accurate/complete checks for the "accelerate to free speed" option in `sc_scenario_helper.get_access_order_implications()`.

Doing the above means I should rerun the deterministic fits at some point.

As a first check though, I should rerun the tests below, as well as my "standard unit tests", to see how much the base model behaviour has been affected.

In [7]:
# append root folder of repo to PYTHONPATH (https://stackoverflow.com/questions/714063/importing-modules-from-parent-folder)
import sys
from pathlib import Path
parent_path = str(Path('.').absolute().parent)
if not (parent_path in sys.path):
    sys.path.append(parent_path)
    sys.path.append(parent_path + '/SCPaper')
print(sys.path)

# imports needed to run the simulations
import math
import copy
import numpy as np
import commotions
import sc_scenario
import sc_scenario_perception
from sc_scenario_helper import CtrlType
import sc_fitting

N_REPS = 100

for model in ('oVAoSNv', 'oVAoAN'):
    print(f'***** Model {model}:')
    assumptions = sc_scenario.get_assumptions_dict_from_string(model)
    params = copy.deepcopy(sc_fitting.DEFAULT_PARAMS)
    params_k = copy.deepcopy(sc_fitting.get_default_params_k(model))
    params.sigma_V = 0.1
    params.tau_theta = 0.02
    for car_ttca in (-3, 8, 80):
        print(f'\t--- Car initial time to conflict area: {car_ttca} s:')
        scenario = sc_fitting.SCPaperScenario('TestScenario', 
                                        initial_ttcas=(3, car_ttca), 
                                        veh_const_speed=True,
                                        stop_criteria = sc_fitting.IN_CS_STOP,
                                        metric_names = ('ped_av_speed_to_CS',),
                                        time_step = sc_fitting.PROB_SIM_TIME_STEP,
                                        end_time = sc_fitting.PROB_SIM_END_TIME)
        vs = np.zeros(N_REPS)
        for i in range(N_REPS):
            sim = sc_fitting.simulate_scenario(scenario, assumptions, params, params_k)
            metrics = sc_fitting.get_metrics_for_scenario(scenario, sim)
            vs[i] = metrics['TestScenario_ped_av_speed_to_CS']
        mean_v = np.mean(vs)
        ci_radius = 1.96 * np.std(vs)/math.sqrt(N_REPS)
        print(f'\t\tAverage ped_av_speed_to_CS: {mean_v:.3f} m/s\n'
              f'\t\tCI: [{mean_v - ci_radius:.3f}, {mean_v + ci_radius:.3f}] m/s')       


['C:\\GITHUB\\COMMOTIONSFramework\\diary notebooks', 'C:\\WinPython\\WPy64-3920\\python-3.9.2.amd64\\python39.zip', 'C:\\WinPython\\WPy64-3920\\python-3.9.2.amd64\\DLLs', 'C:\\WinPython\\WPy64-3920\\python-3.9.2.amd64\\lib', 'C:\\WinPython\\WPy64-3920\\python-3.9.2.amd64', '', 'C:\\WinPython\\WPy64-3920\\python-3.9.2.amd64\\lib\\site-packages', 'C:\\WinPython\\WPy64-3920\\python-3.9.2.amd64\\lib\\site-packages\\win32', 'C:\\WinPython\\WPy64-3920\\python-3.9.2.amd64\\lib\\site-packages\\win32\\lib', 'C:\\WinPython\\WPy64-3920\\python-3.9.2.amd64\\lib\\site-packages\\Pythonwin', 'C:\\WinPython\\WPy64-3920\\python-3.9.2.amd64\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\tragma\\.ipython', 'C:\\GITHUB\\COMMOTIONSFramework', 'C:\\GITHUB\\COMMOTIONSFramework/SCPaper']
***** Model oVAoSNv:
	--- Car initial time to conflict area: -3 s:
		Average ped_av_speed_to_CS: 1.300 m/s
		CI: [1.300, 1.300] m/s
	--- Car initial time to conflict area: 8 s:
		Average ped_av_speed_to_CS: 1.011 m/s
